In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
from datetime import datetime

import tensorflow as tf

if not os.path.exists('facenet'):
    !git clone https://github.com/davidsandberg/facenet --depth=1
        
sys.path.insert(0, './facenet/src')
import align.detect_face
import facenet
import scipy.spatial.distance as distance

In [2]:
model_dir = '/home/sabrine/notebook/reid/20170216-091149/'

In [3]:
# starting a session
tf.reset_default_graph()

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.allow_soft_placement = True

sess = tf.Session(config=tf_config)

print('loading the detection/alignment network...')
pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
print('done!')

print('loading the embedding network...')
meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
restorer = tf.train.import_meta_graph(os.path.join(model_dir, meta_file))
restorer.restore(sess, os.path.join(model_dir, ckpt_file))
print('done!')

# getting input / output tensors
g = tf.get_default_graph()
images_in = g.get_tensor_by_name('input:0')
phase_train_in = g.get_tensor_by_name('phase_train:0')
embeddings = g.get_tensor_by_name('embeddings:0')


loading the detection/alignment network...
done!
loading the embedding network...
done!


In [4]:
imageRGBpath= "/home/sabrine/notebook/BIWI_dataset/Training/036/036_000130-a_16388083_rgb.jpg"

In [6]:
from IPython.display import Image, display

# detection parameters
minsize = 20 
threshold = [ 0.6, 0.7, 0.7 ]
factor = 0.709
             
boxes = []
keypoints = []
images = []
print('detecting faces...')

#display(Image(path))
img = scipy.misc.imread(imageRGBpath))
bbs, kps = align.detect_face.detect_face(img, minsize, 
                                           pnet, rnet, onet, threshold, factor)
kps = np.asarray(kps)
print(kps.shape)
kps = kps.reshape([2,5,-1]).T
if len(kps) != 0:  
    images.append(img)  
    boxes.append(bbs)
    keypoints.append(kps)
print('done!')  

detecting faces...


IOError: [Errno 21] Is a directory: '/home/sabrine/notebook/BIWI_dataset/Training'

In [ ]:
print('crop image to face only')
size = 160
margin = 32

faces = []

for img, bbs in zip(images, boxes):
    img_faces = []
    for x0,y0,x1,y1,_ in bbs.astype(np.int32):
        x0 = np.maximum(x0 - margin//2, 0)
        y0 = np.maximum(y0 - margin//2, 0)
        x1 = np.minimum(x1 + margin//2, img.shape[1])
        y1 = np.minimum(y1 + margin//2, img.shape[0])
        img_faces.append(scipy.misc.imresize(img[y0:y1,x0:x1], (size, size)))
 
print('done')

In [ ]:
imageDpath = "/home/sabrine/notebook/BIWI_dataset/Training/036/036_000130-b_16388045_depth.pgm"

In [ ]:
import re
import numpy

def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as numpy array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return numpy.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))

In [ ]:
image = read_pgm(imageDpath, byteorder='<')

In [ ]:
def rgbToD(y, x):
    return(y*480/960, x*640/1280)

In [ ]:
y0d, x0d = rgbToD(y0, x0)
y1d, x1d = rgbToD(y1, x1)

In [ ]:
ool_fill = np.zeros(image.shape)

def floodFill(y,x, image, i):
    toFill = set()
    toFill.add((y,x))
    while toFill:
        (x,y) = toFill.pop()
        depth = image[y][x]
        if depth > 27320:
            i = i + 1
            if i%10000 == 0:
                print(i)
            fool_fill[y][x] = image[y][x]
            toFill.add((y-1,x))
            toFill.add((y+1,x))
            toFill.add((y,x-1))
            toFill.add((y,x+1))

In [ ]:
floodFill((y1d-y0d)/2, (x1d-x0d)/2, image, 0)

In [ ]:
plt.imshow(fool_fill)